# Performing set operations

Users often call for a way to concatenate results into a single list. In this tutorial, work through some examples of different techniques that are available.

## Prerequisites

This tutorial works with Druid 26.0.0 or later.

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).

<details><summary>    
<b>Run without Docker Compose</b>    
</summary>

If you do not use the Docker Compose environment, you need the following:

* A running Druid instance.
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.
* [matplotlib](https://matplotlib.org/), a library for creating visualizations in Python,
* [pandas](https://pandas.pydata.org/), a data analysis and manipulation tool.
* Jupyter notebook or Jupyter Lab. See [jupyter.org](https://jupyter.org/) for installation instructions.

</details>

### Initialization

Run the next cell to attempt a connection to Druid services. If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Load example flight data

Once your Druid environment is up and running, ingest the sample data for this tutorial.

Open the Druid console, and ingest the data as follows:

1. Load data
2. Batch - SQL
3. Example data
4. Select "FlightCarrierOnTime (1 month)"

For the purposes of this notebook, use all the defaults suggested by the console, including the default datasource name: 

`On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11`

When this is completed, run the following cell for the final part of the initialization. This will provide us some methods to call as we explore what TopN does.

In [ ]:
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## Concatenating result sets with `UNION ALL`

Execute the following query to combine together two different queries - one that contains 10 flights taking off from San Fransisco at around 11 o'clock in the morning, and another with flights departing from Atlanta in the same hour.

In [ ]:
sql = '''
WITH
set1 AS (
  SELECT
  __time,
  "Origin",
  "Tail_Number",
  "Flight_Number_Reporting_Airline"
  FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
  WHERE Origin = 'SFO'
  AND DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
  ORDER BY __time
  LIMIT 10
  ),
set2 AS (
SELECT
  __time,
  "Origin",
  "Tail_Number",
  "Flight_Number_Reporting_Airline"
  FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
  WHERE Origin = 'ATL'
  AND DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
  ORDER BY __time
  LIMIT 10
  )
  
SELECT * from set1
UNION ALL
SELECT * from set2
'''

display.sql(sql)

This is what's known as a [top-level](https://druid.apache.org/docs/latest/querying/sql.html#top-level) `UNION` operation: each set of results was gathered individually, one after the other, and the list of results concatenated.

Notice that these results are not in order by time – even though the individual sets did `ORDER BY` time. Druid has simply concatenated the two result sets together.

The following cell contains another valid use of `UNION ALL`, where some filtering and `GROUP BY` operations have been added. Notice there is a mix of this being done in the sets themselves and as part of the top level `UNION`.

In [ ]:
sql='''
WITH
set1 AS (
    SELECT "Reporting_Airline", "Distance"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
  ),
set2 AS (
    SELECT
        "Reporting_Airline",
        COUNT(*) AS "Frights",
        MAX(Distance) AS "Lengthiest",
        MIN(Distance) AS "Shortest"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-02 11:00:00'
    AND "Reporting_Airline" LIKE 'A%'
    GROUP BY 1
  )

SELECT "Reporting_Airline",
        COUNT(*) AS "Frights",
        MAX(Distance) AS "Lengthiest",
        MIN(Distance) AS "Shortest" from set1
  WHERE "Reporting_Airline" LIKE 'AA'
  GROUP BY 1
UNION ALL
SELECT * from set2
'''

display.sql(sql)

Optionally, run the next cell – it shows the precise `EXPLAIN PLAN` for the query. You can see there are two `query` exeuction plans, one for each of our queries, and that Druid's planning process has taken time to optimize how the query above will actually execute.

In [ ]:
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

The next cell contains two result sets: `set1` provides some statistics for the 1st November, `set2` for the 2nd November.

The `UNION ALL` operation simply concatenates the two sets of results.

In [ ]:
sql='''
WITH
set1 AS (
    SELECT
        "Reporting_Airline",
        COUNT(*) AS "Flights",
        MIN(Distance) AS "Shortest",
        MAX(Distance) AS "Longest"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
    AND "Reporting_Airline" LIKE 'A%'
    GROUP BY 1
  ),
set2 AS (
    SELECT
        "Reporting_Airline",
        COUNT(*) AS "Frights",
        MAX(Distance) AS "Lengthiest",
        MIN(Distance) AS "Shortest"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-02 11:00:00'
    AND "Reporting_Airline" LIKE 'A%'
    GROUP BY 1
  )

SELECT * from set1
UNION ALL
SELECT * from set2
'''

display.sql(sql)

As expected, we see a concatenated set of results - but wait! Why are the longest and shortest the wrong way around?!

As the `UNION ALL` concatenated the sets, it very simply added the results for Atlanta to the end of the results for San Francisco. It did not take into account that the columns in set 2 were in a different order, nor did it take into account _either_ of the errors in field names.

Instead, the query against `set2` ought to have been more explicit, taking into account the proper field names (`flights` vs `frights`!) and giving the same order as the first set of results - ie, `set1`'s schema:

```sql
SELECT "Flights", "Shortest", "Longest" from set1
UNION ALL
SELECT "Frights", "Shortest", "Lengthiest" from set2
```

Run the cell below to see what difference this makes:

In [ ]:
sql='''
WITH
set1 AS (
    SELECT
        "Reporting_Airline",
        COUNT(*) AS "Flights",
        MIN(Distance) AS "Shortest",
        MAX(Distance) AS "Longest"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
    AND "Reporting_Airline" LIKE 'A%'
    GROUP BY 1
  ),
set2 AS (
    SELECT
        "Reporting_Airline",
        COUNT(*) AS "Frights",
        MAX(Distance) AS "Lengthiest",
        MIN(Distance) AS "Shortest"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-02 11:00:00'
    AND "Reporting_Airline" LIKE 'A%'
    GROUP BY 1
  )

SELECT "Flights", "Shortest", "Longest" from set1
UNION ALL
SELECT "Frights", "Shortest", "Lengthiest" from set2
'''

display.sql(sql)

## Working with concatenated result sets

So far we have seen "[top level](https://druid.apache.org/docs/26.0.0/querying/sql.html#top-level)" UNIONs in operation. That is, the `UNION ALL` is being used very simply at the highest level of the query.

When we want to apply functions to the _entire_ result set like above, such as using it in a sub-query or a `FROM` clause, Druid changes how it plans and executes the query. Instead of a top level `UNION ALL`, it switches to a "[table level](https://druid.apache.org/docs/26.0.0/querying/sql.html#table-level)" query.  For example, to put our very first cell results into `__time` order, we might want to use a query that applies `ORDER BY` to a `UNION` of `set1` (San Francisco flights) and `set2` (Atlanta flights), like the following:

```sql
SELECT "Origin",
    "Tail_Number",
    "Flight_Number_Reporting_Airline"
FROM (
    SELECT * from set1
    UNION ALL
    SELECT * from set2
    )
ORDER BY __time
```

Run the cell below to see what happens.

In [ ]:
sql = '''
WITH
set1 AS (
  SELECT
  __time,
  "Origin",
  "Tail_Number",
  "Flight_Number_Reporting_Airline"
  FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
  WHERE Origin = 'SFO'
  AND DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
  ),
set2 AS (
SELECT
  __time,
  "Origin",
  "Tail_Number",
  "Flight_Number_Reporting_Airline"
  FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
  WHERE Origin = 'ATL'
  AND DATE_TRUNC('HOUR', __time) = TIMESTAMP '2005-11-01 11:00:00'
  )

SELECT __time,
    "Origin",
    "Tail_Number",
    "Flight_Number_Reporting_Airline"
FROM (
    SELECT * from set1
    UNION ALL
    SELECT * from set2
    )
ORDER BY __time
'''

display.sql(sql)

Notice the `ClientError` that Druid returns. Rather than simply concatenating two `TABLE`s of results together, Druid switched to table level `UNION ALL`, and with it a switch to [`UNION`](https://druid.apache.org/docs/26.0.0/querying/datasource.html#union) datasources. These cannot be used in the same way as `TABLE`s; there are constraints on what can be done with them. An `EXPLAIN PLAN` for these types of queries shows this in detail:

```json
"type": "union",
"dataSources": [
    {
    "type": "table",
    "name": "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    },
    {
    "type": "table",
    "name": "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    }
```

In large deployments of Druid, data engineers may decide to break up a single data set into multiple `TABLE`s. Some reasons for this might be:

* `TABLE`s separated by their `__time` granularity (ie. the level of roll-up)
* `TABLE`s logically separated for security reasons by tenant
* `TABLE`s broken up by different filter conditions
* `TABLE`s separated by upstream data source (e.g. different Kafka topics for different cloud regions, different groups of source device, and so on)
* `TABLE`s that break the entire data set up into different periods of time, perhaps with different retention periods

The next two cells to create two new tables, `wikipedia-en` and `wikipedia-fr`. One table contains only English language channels, while the second only contains French language channels. Imagine that this is a design decision taken by a data engineer so that they can be governed separately. But now we want to have a single view of both that people can query.

Run these ingestion jobs, and monitor them as they run in the Druid Console.

In [ ]:
sql='''
REPLACE INTO "wikipedia-en" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "isRobot",
  "channel",
  "flags",
  "isUnpatrolled",
  "page",
  "diffUrl",
  "added",
  "comment",
  "commentLength",
  "isNew",
  "isMinor",
  "delta",
  "isAnonymous",
  "user",
  "deltaBucket",
  "deleted",
  "namespace",
  "cityName",
  "countryName",
  "regionIsoCode",
  "metroCode",
  "countryIsoCode",
  "regionName"
FROM "ext"
WHERE "channel" LIKE '#en%'
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('wikipedia-en')
display.table('wikipedia-en')

In [ ]:
sql='''
REPLACE INTO "wikipedia-fr" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "isRobot",
  "channel",
  "flags",
  "isUnpatrolled",
  "page",
  "diffUrl",
  "added",
  "comment",
  "commentLength",
  "isNew",
  "isMinor",
  "delta",
  "isAnonymous",
  "user",
  "deltaBucket",
  "deleted",
  "namespace",
  "cityName",
  "countryName",
  "regionIsoCode",
  "metroCode",
  "countryIsoCode",
  "regionName"
FROM "ext"
WHERE "channel" LIKE '#fr%'
PARTITIONED BY DAY
'''

sql_client.run_task(sql)
sql_client.wait_until_ready('wikipedia-fr')
display.table('wikipedia-fr')

In the next cell, we declare our `UNION ALL` data source as the single, unified view of all the data. Then we use that in our outer `SELECT` to count the number of robot and non-robot edits by channel.

Remember, the `SELECT` in the `unifiedSource` must be simple in order to meet the constraints set by a table level `UNION ALL`, so any filtering ans so on can only be done in the outer `SELECT` statement.

In [ ]:
sql = '''
WITH unifiedSource AS (
    SELECT
        "__time",
        "isRobot",
        "channel",
        "user",
        "countryName"
    FROM "wikipedia-en"
    UNION ALL
    SELECT
        "__time",
        "isRobot",
        "channel",
        "user",
        "countryName"
    FROM "wikipedia-fr"
    )

SELECT
    "channel",
    COUNT(*) FILTER (WHERE isRobot=true) AS "Robot Edits",
    COUNT (DISTINCT user) FILTER (WHERE isRobot=true) AS "Robot Editors",
    COUNT(*) FILTER (WHERE isRobot=false) AS "Human Edits",
    COUNT (DISTINCT user) FILTER (WHERE isRobot=false) AS "Human Editors"
FROM unifiedSource
GROUP BY 1
'''

display.sql(sql)

## Conclusion

* There are two modes for `UNION ALL` in Druid - top level and table level
* Top level is a simple concatenation, and operations must be done on the source `TABLE`s
* Table level uses a `union` data source, and operations must be done on the outer `SELECT`

## Learn more

* Watch [Plan your Druid table datasources](https://youtu.be/OpYDX4RYLV0?list=PLDZysOZKycN7MZvNxQk_6RbwSJqjSrsNR) by Peter Marshall
* Read [union](https://druid.apache.org/docs/26.0.0/querying/datasource.html#union) data sources in the documentation
* Read the latest [documentation](https://druid.apache.org/docs/26.0.0/querying/sql.html#union-all) on the `UNION ALL` operator